## LSTM with validation set

### Imports

In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
import keras
import tensorflow as tf
from keras.layers import LSTM
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


Using TensorFlow backend.


### Load data

In [2]:
df1 = pd.read_json("./data/Sarcasm_Headlines_Dataset.json", lines=True)
df2 = pd.read_json("./data/Sarcasm_Headlines_Dataset_v2.json", lines=True)
# re-order attibute columns in df2
df2 = df2[['article_link','headline','is_sarcastic']]
df = pd.concat([df1, df2], axis=0)
df = df.drop(['article_link'], axis=1)
print(len(df))
df.head()

55328


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


Reset the index as we have merged two different indexes.

In [3]:
df.reset_index(inplace=True, drop=True)

In [4]:
df['len'] = df['headline'].apply(lambda x: len(x.split(" ")))
print(df['len'].mean())
print(min(df['len']))
print(max(df['len']))

9.953368999421631
2
151


### Split data in train, test and val
50% for train, 25% for test, 25% for validation.

In [5]:
train, test = train_test_split(df, test_size=0.25)
print(train.shape)
train, val = train_test_split(train, test_size=0.33333)

(41496, 3)


In [6]:
print('train:',train.shape)
print('test:',test.shape)
print('val:',val.shape)

train: (27664, 3)
test: (13832, 3)
val: (13832, 3)


### Tokenize
Fit the tokenizer only on the text of training data.
Then, we use that same tokenizer to transform the texts of train, val and test sets to sequences of integers.

It's possible to fit on the entire data. But it's probably a better idea to reserve a token for "unknown" words (oov_token=True), for the cases when you find new test data with words your model has never seen.

In [7]:
max_features = 10000 # max num words
maxlen = 25 # we could also try 151
embedding_size = 200

# create the tokenizer with the maximum number of words to keep, 
# based on word frequency. 
# Only the most common num_words-1 words will be kept.
tokenizer = Tokenizer(num_words=max_features, oov_token = True)

# fit the tokenizer on the headlines
tokenizer.fit_on_texts(list(train['headline']))

# Transforms each text in texts to a sequence of integers.
train_X = tokenizer.texts_to_sequences(train['headline'])
test_X = tokenizer.texts_to_sequences(test['headline'])
val_X = tokenizer.texts_to_sequences(val['headline'])

# transforms a list of num_samples sequences (lists of integers)
# into a 2D Numpy array of shape (num_samples, num_timesteps).
train_X = pad_sequences(train_X, maxlen = maxlen)
test_X = pad_sequences(test_X, maxlen = maxlen)
val_X = pad_sequences(val_X, maxlen = maxlen)

train_y = train['is_sarcastic']
test_y = test['is_sarcastic']
val_y = val['is_sarcastic']

Load glove embedding set, construct embedding matrix for words in word_index:


In [8]:
# load embeddings
EMBEDDING_FILE = './embeddings/glove.6B.200d.txt'

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

# first, build index mapping words in the embeddings set
# to their embedding vector
embeddings_index = {}
with open(EMBEDDING_FILE, encoding="utf8") as f:
    for line in f:
        word, coefs = get_coefs(*line.split(" "))
        embeddings_index[word] = coefs
            
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

# Random embedding vector for unknown words.
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
# prepare embedding matrix
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        # words not found in embedding index will be random
        embedding_matrix[i] = embedding_vector

C:\Users\elsas\Anaconda3\envs\TFM\lib\site-packages\IPython\core\interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


### LSTM Model
Model Parameters:

- **Activation Function**: I have used ReLU as the activation function. ReLU is a non-linear activation function, which helps complex relationships in the data to be captured by the model.

- **Optimiser**: We use adam optimiser, which is an adaptive learning rate optimiser.

- **Loss function**: We will train a network to output a probability over the 2 classes using Sigmoid Loss.

In [9]:
# Create model structure

model = Sequential()
model.add(Embedding(max_features, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives()])

### Save the best model and early stopping
To prevent the model from overfitting I have enabled early stopping.

Early stopping is a method that allows us to specify an arbitrary large number of training epochs and stop training once the model performance stops improving on a hold out/validation dataset.


In [10]:
# Save the model after every epoch.
saveBestModel = keras.callbacks.ModelCheckpoint('./model/best_model.hdf5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# Stop training when a monitored quantity has stopped improving.
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [11]:
# Fit the model
batch_size = 100
epochs = 25
model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, validation_data=(val_X, val_y), callbacks=[saveBestModel, earlyStopping])

C:\Users\elsas\Anaconda3\envs\TFM\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 27664 samples, validate on 13832 samples
Epoch 1/25
27664/27664 [==============================] - 57s 2ms/step - loss: 0.5658 - accuracy: 0.6983 - precision_1: 0.7017 - recall_1: 0.5904 - true_positives_1: 7460.0000 - val_loss: 0.3389 - val_accuracy: 0.8558 - val_precision_1: 0.8222 - val_recall_1: 0.8742 - val_true_positives_1: 5537.0000
Epoch 2/25
  100/27664 [..............................] - ETA: 48s - loss: 0.3532 - accuracy: 0.9200 - precision_1: 0.9123 - recall_1: 0.9455 - true_positives_1: 52.0000

C:\Users\elsas\Anaconda3\envs\TFM\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


27664/27664 [==============================] - 55s 2ms/step - loss: 0.3198 - accuracy: 0.8861 - precision_1: 0.8537 - recall_1: 0.9057 - true_positives_1: 11444.0000 - val_loss: 0.2818 - val_accuracy: 0.8879 - val_precision_1: 0.8887 - val_recall_1: 0.8633 - val_true_positives_1: 5468.0000 0.9047 - true_positives - ETA: 2s - loss: 0.3204 - accuracy: 0.8852 - precision_1: 0.8526 - recall_1: 0.9054 - true_positives_1: 1093 - ETA: 1s - loss: 0.3199 - accuracy: 0.8851 - precision_1: 0.8528 - recall_1: 0.9050 - true_pos
Epoch 3/25
27664/27664 [==============================] - 59s 2ms/step - loss: 0.2075 - accuracy: 0.9337 - precision_1: 0.9103 - recall_1: 0.9482 - true_positives_1: 11980.0000 - val_loss: 0.2792 - val_accuracy: 0.9039 - val_precision_1: 0.8962 - val_recall_1: 0.8937 - val_true_positives_1: 5661.0000
Epoch 4/25
27664/27664 [==============================] - 62s 2ms/step - loss: 0.1359 - accuracy: 0.9581 - precision_1: 0.9435 - recall_1: 0.9662 - true_positives_1: 12208.0000 

Stops in the Epoch 6 out of 25, this is thanks to the validation set, that prevents us to overfit the model.

### Evaluate model results with test data

In [12]:
model.metrics_names

['loss', 'accuracy', 'precision_1', 'recall_1', 'true_positives_1']

In [13]:
loss, accuracy, precision, recall, true_positives = model.evaluate(test_X, test_y, batch_size=batch_size)

13832/13832 [==============================] - 7s 482us/step


In [14]:
mult_pr=precision*recall
sum_pr=precision+recall
div=mult_pr/sum_pr
f1_score=2*div

#### Loss, Accuracy, Precision, Recall and F1

In [15]:
print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)
print('True positives:',true_positives)

Loss: 0.43703812065799846
Accuracy: 0.9235829710960388
Precision: 0.9090211987495422
Recall: 0.9273751974105835
f1 score: 0.9181064778862218
True positives: 5925.0


# Extract FalsePositives and FalseNegatives


We first get the list of predictions. In the confusion matrix it can be observed that the number of True Positives is the same.

In [16]:
pred_y = model.predict_classes(test_X, batch_size=batch_size)

In [17]:
confusion_matrix(test_y, pred_y)

array([[6850,  593],
       [ 464, 5925]], dtype=int64)

We build a function to compare the predicted values to the actual values and extract the FalsePositives and FalseNegatives.

In [18]:
def getFP_FN_lists(test_X, test_y, pred_y):
    FP_text = []
    FP_index = []
    FN_text = []
    FN_index = []
    for i in range(len(test_y)):
        if(pred_y[i]==1 and test_y[test_y.index[i]]==0):
            FP_text.append(test['headline'][test_y.index[i]])
            FP_index.append(test_y.index[i])
        elif(pred_y[i]==0 and test_y[test_y.index[i]]==1):
            FN_text.append(test['headline'][test_y.index[i]])
            FN_index.append(test_y.index[i])
            
    return FP_text,FP_index,FN_text,FN_index

In [19]:
'''Returns 2 dataframes, one with all the False Positives and one with all the False Negatives'''
def getFP_FN(test_X, test_y, pred_y):
    FP_text,FP_index,FN_text,FN_index = getFP_FN_lists(test_X, test_y, pred_y)
    d_FP = {'FP_text':FP_text,'FP_index':FP_index}
    df_FP = pd.DataFrame(d_FP)
    d_FN = {'FN_text':FN_text,'FN_index':FN_index}
    df_FN = pd.DataFrame(d_FN)
    
    return df_FP,df_FN

In [20]:
# We get the FPs and FNs as DataFrames and store them to CSVs
df_FP,df_FN = getFP_FN(test_X, test_y, pred_y)
df_FP.to_csv('FP.csv', index=True)
df_FN.to_csv('FN.csv', index=True)